## Importações e Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, col
from pyspark.sql.functions import lower, regexp_replace
import json
import os

# carregando os jars necessários para se conectar ao MinIO (nosso S3)
spark = SparkSession.builder \
    .appName("Read from MinIO S3A and Write Partitioned") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("MINIO_ROOT_USER")) \
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("MINIO_ROOT_PASSWORD")) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.ui.showConsoleProgress", "true") \
    .getOrCreate()

df = spark.read.parquet("s3a://datalake/2_silver/001_breweries/breweries/")

## Lendo da camada silver e jogando na gold em parquet

In [2]:
df.createOrReplaceTempView("breweries")

gold_df = spark.sql("""
    SELECT 
        brewery_type,
        city,
        state_province,
        country,
        COUNT(*) AS brewery_count
    FROM breweries
    GROUP BY brewery_type, city, state_province, country
    ORDER BY brewery_count DESC
""")

gold_df.write.mode("overwrite").parquet("s3a://datalake/3_gold/001_breweries/breweries_by_type_location/")

gold_df.show()

+------------+----------------+--------------+-------------+-------------+
|brewery_type|            city|state_province|      country|brewery_count|
+------------+----------------+--------------+-------------+-------------+
|       micro|          Denver|      Colorado|United States|           58|
|       micro|       San Diego|    California|United States|           45|
|       micro|         Seattle|    Washington|United States|           44|
|       micro|        Portland|        Oregon|United States|           41|
|       micro|          Austin|         Texas|United States|           30|
|     brewpub|        Portland|        Oregon|United States|           29|
|       micro|         Chicago|      Illinois|United States|           28|
|         bar|       Singapore|     Singapore|    Singapore|           28|
|       micro|     Minneapolis|     Minnesota|United States|           25|
|     brewpub|       San Diego|    California|United States|           23|
|       micro|         Ho

## Criando metadados para o assistende de IA

In [3]:
# extraindo colunas
columns = [{"name": field.name, "type": str(field.dataType)} for field in gold_df.schema.fields]

metadata = {
    "table_name": "breweries_by_type_location",
    "columns": columns,
    "data_source": "s3a://datalake/2_silver/001_breweries/breweries/",
    "output_path": "s3a://datalake/3_gold/001_breweries/breweries_by_type_location/",
    "description": "Tabela que contém a contagem de cervejarias por tipo, cidade, estado e país."
}

metadata_path = "gold_001_breweries.json"
with open(metadata_path, 'w') as metadata_file:
    json.dump(metadata, metadata_file, indent=4)